In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('quant.sf', delimiter='\t')
dataST = pd.read_csv('quantST.sf', delimiter='\t')

In [4]:
data

,Name,Length,EffectiveLength,TPM,NumReads
0,ENSMUST00000196221.1,9,1.818,0.0,0.0
1,ENSMUST00000179664.1,11,1.954,0.0,0.0
2,ENSMUST00000177564.1,16,2.119,0.0,0.0
3,ENSMUST00000178537.1,12,2.002,0.0,0.0
4,ENSMUST00000178862.1,14,2.071,0.0,0.0
...,...,...,...,...,...
116541,ENSMUST00000206015.1,934,684.000,0.0,0.0
116542,ENSMUST00000231588.1,967,717.000,0.0,0.0
116543,ENSMUST00000120391.1,1083,833.000,0.0,0.0
116544,ENSMUST00000190779.1,696,446.000,0.0,0.0


In [5]:
data['Name'].values

array(['ENSMUST00000196221.1', 'ENSMUST00000179664.1',
       'ENSMUST00000177564.1', ..., 'ENSMUST00000120391.1',
       'ENSMUST00000190779.1', 'ENSMUST00000230275.1'], dtype=object)

In [6]:
np.where(data['NumReads'] > 100)

(array([   264,    354,    355, ..., 116384, 116430, 116432], dtype=int64),)

In [7]:
new_df = data[data['NumReads'] >= 100]
new_dfST = dataST[dataST['NumReads'] >= 100]


In [8]:
new_df

,Name,Length,EffectiveLength,TPM,NumReads
264,ENSMUST00000106393.7,843,593.0,19.318411,231.450
354,ENSMUST00000112847.8,7196,6946.0,19.419604,2725.246
355,ENSMUST00000141573.7,1502,1252.0,33.381955,844.397
356,ENSMUST00000112846.7,2617,2367.0,8.524482,407.659
358,ENSMUST00000133069.7,2732,2482.0,2.598258,130.291
...,...,...,...,...,...
116165,ENSMUST00000216296.1,1480,1230.0,4.672461,116.113
116281,ENSMUST00000210683.1,2545,2295.0,2.835215,131.462
116384,ENSMUST00000219562.1,774,524.0,17.921242,189.727
116430,ENSMUST00000164420.1,3846,3596.0,9.418782,684.298


In [67]:
new_df.loc[new_df['Name'].str.contains('ENSMUST00000211')]

,Name,Length,EffectiveLength,TPM,NumReads


In [10]:
from pyensembl import EnsemblRelease
import re 

In [11]:
reference = EnsemblRelease(release=102, species='mus_musculus')
def getname(x):
    try:
        return reference.transcript_by_id(re.split(r'\.', x)[0]).transcript_name
    except:
        return x
    
getname_v = np.vectorize(getname)



In [12]:
new_df.loc[new_df['Name'] == 'ENSMUST00000021271.13']



re.split(r'\.', 'ENSMUST00000021271.13')[0]



'ENSMUST00000021271'

In [13]:

new_df['Name'] = new_df['Name'].apply(lambda x: getname_v(x))
new_dfST['Name'] = new_dfST['Name'].apply(lambda x: getname_v(x))

C:\Users\tkoti\AppData\Local\Temp\ipykernel_14412\1723509563.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Name'] = new_df['Name'].apply(lambda x: getname_v(x))
C:\Users\tkoti\AppData\Local\Temp\ipykernel_14412\1723509563.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dfST['Name'] = new_dfST['Name'].apply(lambda x: getname_v(x))


In [14]:
new_dfNT = new_df 
new_dfNT.shape, new_dfST.shape


((21661, 5), (19693, 5))

In [15]:
bothST = pd.merge(new_dfNT, new_dfST, on='Name', how='inner')
bothST.rename(columns={'NumReads_x': 'NumReadsNT', "TPM_x" : "TPM_NT", "TPM_y" : "TPM_ST", 'NumReads_y': 'NumReadsST'}, inplace=True)

In [16]:
bothST

,Name,Length_x,EffectiveLength_x,TPM_NT,NumReadsNT,Length_y,EffectiveLength_y,TPM_ST,NumReadsST
0,Hyi-202,843,593.0,19.318411,231.450,843,593.0,19.991816,185.731
1,Mapk10-203,7196,6946.0,19.419604,2725.246,7196,6946.0,21.416713,2330.592
2,Mapk10-209,1502,1252.0,33.381955,844.397,1502,1252.0,37.450557,734.584
3,Mapk10-202,2617,2367.0,8.524482,407.659,2617,2367.0,9.113561,337.960
4,Mapk10-206,2732,2482.0,2.598258,130.291,2732,2482.0,2.900522,112.786
...,...,...,...,...,...,...,...,...,...
19043,Gm29216-201,492,242.0,13421.021842,65619.291,492,242.0,13564.503123,51427.791
19044,Gm45551-201,928,678.0,12.530011,171.637,928,678.0,12.943543,137.487
19045,Gm2436-201,774,524.0,17.921242,189.727,774,524.0,17.243295,141.557
19046,Gm17131-201,3846,3596.0,9.418782,684.298,3846,3596.0,10.299602,580.255


In [17]:
#ST short day cycle #NT normal day cycle
bothST['TPM_Diff'] = bothST['TPM_NT'] - bothST['TPM_ST']
bothST['gene_name'] = bothST['Name'].apply(lambda x: reference.gene_name_of_transcript_name(x) if not ValueError else x)

In [18]:
bothST

,Name,Length_x,EffectiveLength_x,TPM_NT,NumReadsNT,Length_y,EffectiveLength_y,TPM_ST,NumReadsST,TPM_Diff,gene_name
0,Hyi-202,843,593.0,19.318411,231.450,843,593.0,19.991816,185.731,-0.673405,Hyi-202
1,Mapk10-203,7196,6946.0,19.419604,2725.246,7196,6946.0,21.416713,2330.592,-1.997109,Mapk10-203
2,Mapk10-209,1502,1252.0,33.381955,844.397,1502,1252.0,37.450557,734.584,-4.068602,Mapk10-209
3,Mapk10-202,2617,2367.0,8.524482,407.659,2617,2367.0,9.113561,337.960,-0.589079,Mapk10-202
4,Mapk10-206,2732,2482.0,2.598258,130.291,2732,2482.0,2.900522,112.786,-0.302264,Mapk10-206
...,...,...,...,...,...,...,...,...,...,...,...
19043,Gm29216-201,492,242.0,13421.021842,65619.291,492,242.0,13564.503123,51427.791,-143.481281,Gm29216-201
19044,Gm45551-201,928,678.0,12.530011,171.637,928,678.0,12.943543,137.487,-0.413532,Gm45551-201
19045,Gm2436-201,774,524.0,17.921242,189.727,774,524.0,17.243295,141.557,0.677947,Gm2436-201
19046,Gm17131-201,3846,3596.0,9.418782,684.298,3846,3596.0,10.299602,580.255,-0.880820,Gm17131-201


In [19]:
noexpressiondiff = bothST.loc[np.isclose(bothST['TPM_Diff'], 0, atol = 0.05)]

In [20]:
noexpressiondiff

,Name,Length_x,EffectiveLength_x,TPM_NT,NumReadsNT,Length_y,EffectiveLength_y,TPM_ST,NumReadsST,TPM_Diff,gene_name
27,Tm9sf3-201,8096,7846.0,19.298730,3059.198,8096,7846.0,19.270449,2368.747,0.028281,Tm9sf3-201
71,Gdpd5-201,4611,4361.0,1.705606,150.278,4611,4361.0,1.750578,119.604,-0.044972,Gdpd5-201
99,Exoc7-202,3205,2955.0,5.224697,311.924,3205,2955.0,5.230944,242.168,-0.006247,Exoc7-202
108,Nfatc2-208,5399,5149.0,2.925344,304.320,5399,5149.0,2.878053,232.167,0.047291,Nfatc2-208
136,Golga4-204,4461,4211.0,19.416267,1651.892,4461,4211.0,19.375474,1278.251,0.040793,Golga4-204
...,...,...,...,...,...,...,...,...,...,...,...
18904,1700086L19Rik-203,795,545.0,31.543493,347.326,795,545.0,31.552413,269.406,-0.008920,1700086L19Rik-203
18908,Thap6-201,3151,2901.0,3.300015,193.417,3151,2901.0,3.325159,151.126,-0.025144,Thap6-201
18920,Gm46136-202,6329,6079.0,2.067734,253.955,6329,6079.0,2.093608,199.391,-0.025874,Gm46136-202
18935,Gm10033-204,7778,7528.0,2.120312,322.485,7778,7528.0,2.104627,248.218,0.015685,Gm10033-204


In [74]:
circadiantranscripts = pd.concat([bothST.loc[bothST['Name'].str.contains('Clo')],bothST.loc[bothST['Name'].str.contains('Per')], bothST.loc[bothST['Name'].str.contains('Cry1')], bothST.loc[bothST['Name'].str.contains("Cry2")], bothST.loc[bothST['Name'].str.contains('Rev')], bothST.loc[bothST['Name'].str.contains('Ror')], bothST.loc[bothST['Name'].str.contains('Ciart')], bothST.loc[bothST['Name'].str.contains('Noct')]], axis = 0)

In [82]:
circadiantranscripts = circadiantranscripts.drop(circadiantranscripts.loc[circadiantranscripts['Name'] == "Perm1-201"].index)

In [83]:
circadiantranscripts

,Name,Length_x,EffectiveLength_x,TPM_NT,NumReadsNT,Length_y,EffectiveLength_y,TPM_ST,NumReadsST,TPM_Diff,gene_name
17697,Clock-205,9744,9494.0,15.955333,3060.453,9744,9494.0,16.992567,2527.475,-1.037234,Clock-205
1944,Per2-201,5833,5583.0,33.280976,3754.000,5833,5583.0,33.566790,2936.000,-0.285814,Per2-201
2844,Per1-202,3507,3257.0,8.985934,591.305,3507,3257.0,8.843974,451.278,0.141960,Per1-202
2845,Per1-201,4671,4421.0,35.545186,3174.913,4671,4421.0,43.699196,3026.722,-8.154010,Per1-201
14375,Per3-201,5999,5749.0,9.810271,1139.473,5999,5749.0,8.424317,758.762,1.385954,Per3-201
14376,Per3-204,3663,3413.0,5.049825,348.212,3663,3413.0,6.935182,370.828,-1.885357,Per3-204
14377,Per3-205,4909,4659.0,1.256863,118.307,4909,4659.0,1.401199,102.275,-0.144336,Per3-205
18705,Cry1-201,3026,2776.0,25.627893,1437.351,3026,2776.0,26.120426,1136.000,-0.492533,Cry1-201
13448,Cry2-201,3976,3726.0,45.448065,3421.282,3976,3726.0,47.124887,2750.881,-1.676822,Cry2-201
13285,Rev1-201,4262,4012.0,9.923059,804.335,4262,4012.0,9.870420,620.405,0.052639,Rev1-201


In [1]:
pd.save_csv('gene_expression_data', bothST)

NameError: name 'pd' is not defined